In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from utilities.evaluation import ModelEvaluation
from sklearn.tree import DecisionTreeClassifier
from mlxtend.evaluate import bias_variance_decomp
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score